## 1. Configuración Inicial

In [ ]:
import sys
from pathlib import Path
import torch
import logging

# Add project root to path
project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.insert(0, str(project_root))

from src.detection.models import get_model
from src.detection.trainer import DetectionTrainer
from src.logger_cfg.setup import setup_logger

# Setup logging
logger = setup_logger(
    name='hotspot_detection_training',
    log_file=project_root / 'logs' / 'notebook_training.log',
    level=logging.INFO
)

# Check GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")
if device == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## 2. Configuración de Hiperparámetros

In [ ]:
# Training Configuration
CONFIG = {
    # Model selection
    'model_type': 'yolov11',  # 'yolov8' or 'yolov11'
    'model_size': 's',         # 'n', 's', 'm', 'l', 'x'
    
    # Training parameters
    'epochs': 50,
    'batch_size': 32,          # Adjust based on GPU memory
    'img_size': 640,
    'learning_rate': 0.01,
    
    # Model parameters
    'num_classes': 1,          # Single class: 'hotspot'
    'pretrained': True,
    
    # Training settings
    'early_stopping': 10,
    'save_period': 10,
    'num_workers': 4,
    
    # Paths
    'data_yaml': project_root / 'data' / 'yolo_detection' / 'data.yaml',
    'save_dir': project_root / 'models' / 'hotspot_detection'
}

print("Training Configuration:")
print("="*50)
for key, value in CONFIG.items():
    print(f"{key:20s}: {value}")
print("="*50)

## 3. Entrenar YOLOv11

In [ ]:
# Update config for YOLOv11
CONFIG['model_type'] = 'yolov11'
CONFIG['batch_size'] = 32  # YOLOv11 can handle larger batches

# Create model
print("\n" + "="*60)
print(f"Creating YOLOv11{CONFIG['model_size']} model...")
print("="*60)

model_v11 = get_model(
    model_type=CONFIG['model_type'],
    model_size=CONFIG['model_size'],
    num_classes=CONFIG['num_classes'],
    pretrained=CONFIG['pretrained'],
    device=device
)

# Train
print("\nStarting YOLOv11 training...")
save_dir = CONFIG['save_dir'] / f"yolov11{CONFIG['model_size']}"

results_v11 = DetectionTrainer.train_yolo(
    model=model_v11,
    data_yaml=CONFIG['data_yaml'],
    epochs=CONFIG['epochs'],
    batch_size=CONFIG['batch_size'],
    img_size=CONFIG['img_size'],
    lr0=CONFIG['learning_rate'],
    save_dir=save_dir,
    patience=CONFIG['early_stopping'],
    save_period=CONFIG['save_period'],
    workers=CONFIG['num_workers'],
    verbose=True
)

print("\n" + "="*60)
print("YOLOv11 Training Complete!")
print("="*60)
print(f"Best model saved to: {save_dir / 'weights' / 'best.pt'}")

## 4. Entrenar YOLOv8 (Comparación)

In [ ]:
# Update config for YOLOv8
CONFIG['model_type'] = 'yolov8'
CONFIG['batch_size'] = 32

# Create model
print("\n" + "="*60)
print(f"Creating YOLOv8{CONFIG['model_size']} model...")
print("="*60)

model_v8 = get_model(
    model_type=CONFIG['model_type'],
    model_size=CONFIG['model_size'],
    num_classes=CONFIG['num_classes'],
    pretrained=CONFIG['pretrained'],
    device=device
)

# Train
print("\nStarting YOLOv8 training...")
save_dir = CONFIG['save_dir'] / f"yolov8{CONFIG['model_size']}"

results_v8 = DetectionTrainer.train_yolo(
    model=model_v8,
    data_yaml=CONFIG['data_yaml'],
    epochs=CONFIG['epochs'],
    batch_size=CONFIG['batch_size'],
    img_size=CONFIG['img_size'],
    lr0=CONFIG['learning_rate'],
    save_dir=save_dir,
    patience=CONFIG['early_stopping'],
    save_period=CONFIG['save_period'],
    workers=CONFIG['num_workers'],
    verbose=True
)

print("\n" + "="*60)
print("YOLOv8 Training Complete!")
print("="*60)
print(f"Best model saved to: {save_dir / 'weights' / 'best.pt'}")

## 5. Comparar Resultados

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Read results from CSV files
results_v11_df = pd.read_csv(CONFIG['save_dir'] / 'yolov11s' / 'results.csv')
results_v8_df = pd.read_csv(CONFIG['save_dir'] / 'yolov8s' / 'results.csv')

# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# mAP@50
axes[0, 0].plot(results_v11_df['epoch'], results_v11_df['metrics/mAP50(B)'], 
                label='YOLOv11s', marker='o', linewidth=2)
axes[0, 0].plot(results_v8_df['epoch'], results_v8_df['metrics/mAP50(B)'], 
                label='YOLOv8s', marker='s', linewidth=2)
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('mAP@50')
axes[0, 0].set_title('mAP@50 Comparison')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# mAP@50-95
axes[0, 1].plot(results_v11_df['epoch'], results_v11_df['metrics/mAP50-95(B)'], 
                label='YOLOv11s', marker='o', linewidth=2)
axes[0, 1].plot(results_v8_df['epoch'], results_v8_df['metrics/mAP50-95(B)'], 
                label='YOLOv8s', marker='s', linewidth=2)
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('mAP@50-95')
axes[0, 1].set_title('mAP@50-95 Comparison')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Precision
axes[1, 0].plot(results_v11_df['epoch'], results_v11_df['metrics/precision(B)'], 
                label='YOLOv11s', marker='o', linewidth=2)
axes[1, 0].plot(results_v8_df['epoch'], results_v8_df['metrics/precision(B)'], 
                label='YOLOv8s', marker='s', linewidth=2)
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Precision')
axes[1, 0].set_title('Precision Comparison')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Recall
axes[1, 1].plot(results_v11_df['epoch'], results_v11_df['metrics/recall(B)'], 
                label='YOLOv11s', marker='o', linewidth=2)
axes[1, 1].plot(results_v8_df['epoch'], results_v8_df['metrics/recall(B)'], 
                label='YOLOv8s', marker='s', linewidth=2)
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Recall')
axes[1, 1].set_title('Recall Comparison')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(project_root / 'models' / 'hotspot_detection' / 'model_comparison.png', 
            dpi=150, bbox_inches='tight')
plt.show()

# Print best metrics
print("\n" + "="*60)
print("Best Metrics Comparison")
print("="*60)

print("\nYOLOv11s:")
print(f"  mAP@50:     {results_v11_df['metrics/mAP50(B)'].max():.4f}")
print(f"  mAP@50-95:  {results_v11_df['metrics/mAP50-95(B)'].max():.4f}")
print(f"  Precision:  {results_v11_df['metrics/precision(B)'].max():.4f}")
print(f"  Recall:     {results_v11_df['metrics/recall(B)'].max():.4f}")

print("\nYOLOv8s:")
print(f"  mAP@50:     {results_v8_df['metrics/mAP50(B)'].max():.4f}")
print(f"  mAP@50-95:  {results_v8_df['metrics/mAP50-95(B)'].max():.4f}")
print(f"  Precision:  {results_v8_df['metrics/precision(B)'].max():.4f}")
print(f"  Recall:     {results_v8_df['metrics/recall(B)'].max():.4f}")
print("="*60)

## 6. Limpiar Memoria GPU

In [ ]:
if device == 'cuda':
    torch.cuda.empty_cache()
    print("GPU memory cleared")